<a href="https://colab.research.google.com/github/SinaLab/ImageEvalSharedTask2025/blob/main/ImageVal_ZeroShot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 61.6 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch
import os
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load processor and model
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# Define folder path for images
image_folder = "/content/drive/MyDrive/ImageVal/Test/images"
output_csv = "/content/drive/MyDrive/ImageVal/Test/Test_dataset_captions_nofinetune_2nd_round.csv"

# Prepare CSV file for writing
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["image_file", "arabic_caption"])

    # Process each image in the folder
    for image_file in os.listdir(image_folder):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, image_file)
            try:
                image = Image.open(image_path)
                messages = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "image", "image": image},
                            {
                                "type": "text",
                                "text": (
                                     "You are an expert in visual scene understanding and multilingual caption generation."
                                     "Analyze the content of this image, which is potentially related to the palestnian Nakba"
                                     "and Israeli occupation of Palestine, and provide a concise and meaningful caption in Arabic - about 15 to 50 words."
                                     "The caption should reflect the scene's content, emotional context, and should be natural and culturally appropriate."
                                     " Do not include any English or metadata — The caption must be in Arabic."
                                ),
                            },
                        ],
                    }
                ]

                text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
                image_inputs, _ = process_vision_info(messages)
                inputs = processor(
                    text=[text],
                    images=image_inputs,
                    padding=True,
                    return_tensors="pt",
                )
                inputs = inputs.to(model.device)

                with torch.no_grad():
                    generated_ids = model.generate(**inputs, max_new_tokens=128)

                generated_ids_trimmed = [
                    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
                ]
                output_text = processor.batch_decode(
                    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
                )

                print(f"{image_file}: {output_text[0]}")
                writer.writerow([image_file, output_text[0]])

            except Exception as e:
                print(f"Failed to process {image_file}: {e}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Failed to process ISH.PH03.05B.092.jpg: CUDA out of memory. Tried to allocate 1.15 GiB. GPU 0 has a total capacity of 14.74 GiB of which 482.12 MiB is free. Process 22656 has 14.27 GiB memory in use. Of the allocated memory 13.33 GiB is allocated by PyTorch, and 834.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Failed to process ISH.PH03.05B.057.jpg: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 0 has a total capacity of 14.74 GiB of which 496.12 MiB is free. Process 22656 has 14.25 GiB memory in use. Of the allocated memory 13.15 GiB is allocated by PyTorch, and 1004.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.